In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [3]:
# Load the dataset
data = pd.read_csv("../Datasets/BTC-USD.csv")

df = pd.DataFrame(data[0:-1])

df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-05-20,26888.841797,27155.158203,26843.277344,27129.585938,27129.585938,7044911360
1,2023-05-21,27118.423828,27265.917969,26706.921875,26753.826172,26753.826172,8647416921
2,2023-05-22,26749.892578,27045.734375,26549.734375,26851.277344,26851.277344,11056770492
3,2023-05-23,26855.960938,27434.683594,26816.179688,27225.726563,27225.726563,13697203143
4,2023-05-24,27224.603516,27224.603516,26106.576172,26334.818359,26334.818359,16299104428
...,...,...,...,...,...,...,...
361,2024-05-15,61553.988281,66454.453125,61330.410156,66267.492188,66267.492188,39815167074
362,2024-05-16,66256.109375,66712.429688,64613.054688,65231.582031,65231.582031,31573077994
363,2024-05-17,65231.296875,67459.460938,65119.316406,67051.875000,67051.875000,28031279310
364,2024-05-18,67066.210938,67387.328125,66663.500000,66940.804688,66940.804688,16712277406


In [ ]:
# Prepare Data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df["Close"].values.reshape(-1, 1))

scaled_data

array([[4.18048230e-02],
       [3.39697734e-02],
       [3.60017492e-02],
       [4.38094723e-02],
       [2.52329493e-02],
       [2.81810756e-02],
       [3.32496733e-02],
       [3.63578083e-02],
       [6.17398519e-02],
       [5.46554055e-02],
       [5.37476444e-02],
       [4.36829396e-02],
       [3.53490083e-02],
       [4.43070508e-02],
       [4.06693274e-02],
       [4.15854779e-02],
       [1.32493214e-02],
       [4.40817192e-02],
       [2.54660594e-02],
       [2.88485183e-02],
       [2.82679828e-02],
       [1.51497553e-02],
       [1.70040071e-02],
       [1.62185840e-02],
       [1.65569684e-02],
       [0.00000000e+00],
       [9.41888708e-03],
       [2.50795788e-02],
       [2.88997911e-02],
       [2.52620270e-02],
       [3.59965771e-02],
       [6.67825484e-02],
       [1.02225632e-01],
       [9.98274154e-02],
       [1.16157830e-01],
       [1.13097425e-01],
       [1.11670501e-01],
       [1.07309868e-01],
       [1.16005519e-01],
       [1.03454795e-01],


In [ ]:
prediction_days = 300

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
# Defining stochastic Activation Function

class StochasticActivation(tf.keras.layers.Layer):
    def __init__(self, gamma, reaction_func, noise_func, activation="relu", **kwargs):
        super(StochasticActivation, self).__init__(**kwargs)
        self.gamma = gamma
        self.reaction_func = reaction_func
        self.activation = tf.keras.activations.get(activation)  # Get activation function by name
        self.noise_func = noise_func 

    def call(self, inputs, prev_state=None):
        # Apply the formula
        if prev_state is None:
            prev_state = tf.zeros_like(inputs)  # Initialize for first state
        ht = self.activation(inputs) 
        xi_t = self.noise_func(tf.shape(inputs))  # Sample random variable
        st = ht + self.gamma * xi_t * self.reaction_func(ht, prev_state)
        return st  # Return current and updated state

In [ ]:
# Define Noise Function (Example: Gaussian Noise)
def noise_func(shape):
  return tf.random.normal(shape=shape, mean=0.0, stddev=0.6)  # Adjust stddev for noise level

In [ ]:
# Define Reaction Function (Example: Identity Function)
def reaction_func(ht, prev_state):
  return ht - prev_state  # Adjust this function as needed (e.g., scaling factor)

In [ ]:
# Building the Model
model = Sequential()
model.add(Dense(150, activation="relu", input_shape=(prediction_days, 1)))
model.add(Dense(130, activation="relu", input_shape=(prediction_days, 1)))
model.add(Dense(100, activation="relu", input_shape=(prediction_days, 1)))
model.add(Dense(50, activation="relu", input_shape=(prediction_days, 1)))
model.add(Dense(25, activation="relu", input_shape=(prediction_days, 1)))
model.add(Dense(10, activation="relu", input_shape=(prediction_days, 1)))
model.add(StochasticActivation(gamma=0.12, reaction_func=reaction_func, noise_func=noise_func))  # Adjust gamma for noise impact
model.add(Dense(1, activation='linear'))

d:\My Drive\University\Masters\Deep Neural Network\Presentation\Implementation\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the Model
model.compile(optimizer="adam", loss="mse")

In [ ]:
# Train the Model
model.fit(x_train, y_train, epochs=120, batch_size=32)

Epoch 1/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.7296
Epoch 2/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6664 
Epoch 3/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6227 
Epoch 4/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5620 
Epoch 5/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4862 
Epoch 6/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3920 
Epoch 7/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2808 
Epoch 8/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1584 
Epoch 9/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0901 
Epoch 10/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1111 
Epoch 11/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1146
Epoch 12/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0749 
Epoch 13/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0552 
Epoch 14/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0532 
Epoch 15/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0519 
Epoch 16/700
3/3 ━━━

In [ ]:
future_price = len(scaled_data)
actual_price = data.iloc[future_price, 4]
print(f"Actual Price for Day {future_price}: ${actual_price:.2f}")

Actual Price for Day 366: $67282.47


In [ ]:
future_price = len(scaled_data)

predictions = []
absolute_percentage_errors = []

for i in range(100):
    # Invert Scaling for Actual Price Prediction
    sample = scaled_data[future_price-prediction_days:future_price, 0].reshape(1, prediction_days, 1)
    prediction = model.predict(sample)[0, 0]
    predicted_price = scaler.inverse_transform(np.array([prediction]))  # Wrap the prediction in a NumPy array
    predictions.append(predicted_price[0][0])
    # Get the actual value
    actual_value = data.iloc[future_price, 4]

    # Calculate absolute percentage error
    if actual_value != 0:
        absolute_percentage_error = abs((predicted_price[0][0] - actual_value) / actual_value) * 100
        absolute_percentage_errors.append(absolute_percentage_error)

mean_prediction = np.mean(predictions)
print(f"Predicted Price for Day {future_price}: ${mean_prediction:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━

In [ ]:
percentage_difference = abs((mean_prediction - actual_price) / actual_price) * 100
print(f"Absolute Percentage Difference: {percentage_difference:.2f}%")
accuracy = 100 - percentage_difference
print(f"Absolute Percentage Difference: {accuracy:.2f}%")
# Calculate the mean absolute percentage error
mean_absolute_percentage_error = sum(absolute_percentage_errors) / len(absolute_percentage_errors)
print(f"Mean Absolute Percentage Error (MAPE): {mean_absolute_percentage_error:.2f}%")

Absolute Percentage Difference: 0.94%
Absolute Percentage Difference: 99.06%
Mean Absolute Percentage Error (MAPE): 1.89%
